# K-means to segment Customers based on RFM Variables

In [1]:
# อิติปาระมิตา ติงสา อิติสัพพัญญะ มาคะตา
# อิติโพธิ มุนุปปัตโต อิติปิโส จะตะมะโน
# นะเมตตา โมกรุณา พุทธปราณี ธายินดี
# ยะเอ็นดู ยะหันตวา ธามัวเมา พุทพาเอา นะโมพุทธายะ
%matplotlib widget

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans

import plotly.offline as pyoff
import plotly.graph_objs as go

import datetime as dt
import feature_engine

from feature_engine.outliers import Winsorizer
from mpl_toolkits.mplot3d import Axes3D
import plotly
import plotly.graph_objs as go

import warnings
warnings.filterwarnings("ignore")




In [2]:
# Load Dataset
df = pd.read_csv('Data 3 Shops OCT 2021-JAN 2022.csv')

df = df[['รหัสลูกค้า','ช่องทาง','CalendarDay','Week','Site_Key','Site_MediumText','Product_Champion','Article_Key','Article','Sales_Excld','Recency']]

df = df[df['Product_Champion'].isin(['Egg', 'Other', 'Poultry', 'Process Food', 'Process Meat', 'Swine']) & df['Site_Key'].isin(['S035','S154','S031'])]
# 'Egg', 'Other', 'Poultry', 'Process Food', 'Process Meat', 'Swine'
# 'สาขามุกดาหาร' , 'สาขาธาตุพนม' ,'สาขาหนองคาย'

df
# print(df)

,รหัสลูกค้า,ช่องทาง,CalendarDay,Week,Site_Key,Site_MediumText,Product_Champion,Article_Key,Article,Sales_Excld,Recency
0,189,INDUSTRIAL,10/1/2021,9_W5_2021,S154,สาขาธาตุพนม,Swine,1000005614,FR PK BTG หนังหมู A-L 10KG,32500.0,119
1,189,INDUSTRIAL,10/1/2021,9_W5_2021,S154,สาขาธาตุพนม,Swine,1000005609,FR PK BTG หนังหมู A-B 10KG,12720.0,113
2,189,INDUSTRIAL,10/1/2021,9_W5_2021,S154,สาขาธาตุพนม,Swine,1000005616,FR PK BTG หนังหมู A-S 10KG,9000.0,113
3,630,FOOD SERVICE,10/1/2021,9_W5_2021,S035,สาขามุกดาหาร,Swine,1000004760,FR PK BTG สันคอหมู B-4 FZ,2608.8,113
4,277,FOOD SERVICE,10/1/2021,9_W5_2021,S154,สาขาธาตุพนม,Swine,1000005503,FR PK BTG สันนอกหมูสไลด์ 4mm 1KG/K,2300.0,119
...,...,...,...,...,...,...,...,...,...,...,...
31406,408,WHOLESALE,1/31/2022,2_W1_2022,S154,สาขาธาตุพนม,Egg,1000015235,Egg No.3 (Betagro) Pack 150,1305.0,1
31407,121,WHOLESALE,1/31/2022,2_W1_2022,S154,สาขาธาตุพนม,Egg,1000015238,EGG NO.4 (BETAGRO) PACK 150,1215.0,1
31408,193,WHOLESALE,1/31/2022,2_W1_2022,S154,สาขาธาตุพนม,Egg,1000015238,EGG NO.4 (BETAGRO) PACK 150,1215.0,1
31409,402,WHOLESALE,1/31/2022,2_W1_2022,S031,สาขาหนองคาย,Egg,1000005937,Egg X-Tra Large (Betagro) Pack 10,792.0,1


In [3]:
df.tail()

,รหัสลูกค้า,ช่องทาง,CalendarDay,Week,Site_Key,Site_MediumText,Product_Champion,Article_Key,Article,Sales_Excld,Recency
31406,408,WHOLESALE,1/31/2022,2_W1_2022,S154,สาขาธาตุพนม,Egg,1000015235,Egg No.3 (Betagro) Pack 150,1305.0,1
31407,121,WHOLESALE,1/31/2022,2_W1_2022,S154,สาขาธาตุพนม,Egg,1000015238,EGG NO.4 (BETAGRO) PACK 150,1215.0,1
31408,193,WHOLESALE,1/31/2022,2_W1_2022,S154,สาขาธาตุพนม,Egg,1000015238,EGG NO.4 (BETAGRO) PACK 150,1215.0,1
31409,402,WHOLESALE,1/31/2022,2_W1_2022,S031,สาขาหนองคาย,Egg,1000005937,Egg X-Tra Large (Betagro) Pack 10,792.0,1
31410,267,WHOLESALE,1/31/2022,2_W1_2022,S031,สาขาหนองคาย,Egg,1000005937,Egg X-Tra Large (Betagro) Pack 10,340.0,1


In [4]:
df.shape

(31411, 11)

## Data Preparation

In [5]:
def summary(df, pred=None):
  obs = df.shape[0]
  types = df.dtypes
  counts = df.apply(lambda x: x.count())
  min = df.min()
  uniques = df.apply(lambda x: x.unique().shape[0])
  nulls = df.apply(lambda x: x.isnull().sum())
  print('Data shape:', df.shape)

  if pred is None:
        cols = ['types', 'counts', 'uniques', 'nulls', 'min']
        str = pd.concat([types, counts, uniques, nulls, min], axis = 1, sort=True)

  str.columns = cols
  dtypes = str.types.value_counts()
  print('___________________________\nData types:')
  print(str.types.value_counts())
  print('___________________________')
  return str

details = summary(df)
display(details.sort_values(by='nulls', ascending=False))

Data shape: (31411, 11)
___________________________
Data types:
object     7
int64      3
float64    1
Name: types, dtype: int64
___________________________


,types,counts,uniques,nulls,min
Article,object,31411,502,0,BA P&C BTG ลูกชิ้นปรุงรส B 500ก
Article_Key,int64,31411,502,0,1000000985
CalendarDay,object,31411,104,0,1/10/2022
Product_Champion,object,31411,6,0,Egg
Recency,int64,31411,104,0,1
Sales_Excld,float64,31411,5385,0,0.0
Site_Key,object,31411,3,0,S031
Site_MediumText,object,31411,3,0,สาขาธาตุพนม
Week,object,31411,19,0,10_W1_2021
ช่องทาง,object,31411,3,0,FOOD SERVICE


In [6]:
df.describe()

,รหัสลูกค้า,Article_Key,Sales_Excld,Recency
count,31411.000000,3.141100e+04,31411.000000,31411.000000
mean,299.582344,1.000073e+09,1851.598059,61.078444
std,196.576451,7.979281e+06,7424.870711,34.955122
min,1.000000,1.000001e+09,0.000000,1.000000
25%,122.000000,1.000005e+09,480.000000,33.000000
50%,267.000000,1.000005e+09,850.000000,61.000000
75%,454.000000,1.000006e+09,1650.000000,90.000000
max,709.000000,2.000000e+09,792000.000000,123.000000


In [7]:
# df.loc[(df.Recency<0)]

In [8]:
df.loc[(df.Sales_Excld<0)]

,รหัสลูกค้า,ช่องทาง,CalendarDay,Week,Site_Key,Site_MediumText,Product_Champion,Article_Key,Article,Sales_Excld,Recency


**Observation:**

- The min and max value for Quantity is 80995, this could represent cancelled or returned orders.
- The UnitPrice also have few negative values which is uncommon,these transactions could represent cancelled orders by customers or bad-debt incurred by the business.
- Bad debt adjustments will be dropped from the dataset as these do not represent actual sales. 

We need to clean the above values by removing from the dataset

In [9]:
df.isnull().mean()*100

รหัสลูกค้า          0.0
ช่องทาง             0.0
CalendarDay         0.0
Week                0.0
Site_Key            0.0
Site_MediumText     0.0
Product_Champion    0.0
Article_Key         0.0
Article             0.0
Sales_Excld         0.0
Recency             0.0
dtype: float64

There are almost 25% missing CustomerID. We need to remove them as there is no way we can get the number of CustomerID

<b>As customer clusters may vary by geography, we will restrict the data to only United Kingdom customers, which contains most of our customers historical data.</b>

We can see that there is maximum proportion (More than 90% of customers) of 'United Kingdom' customers. So we will take 'United Kingdom' customer for our analysis. 

**Removing the negative values from UnitPrice and Quantity**

In [10]:
# df = df[df.Recency > 0]
# df = df[df.Sale_Value > 0]

# #Removing the Null values from the data.
df = df[pd.notnull(df['รหัสลูกค้า'])]
df

,รหัสลูกค้า,ช่องทาง,CalendarDay,Week,Site_Key,Site_MediumText,Product_Champion,Article_Key,Article,Sales_Excld,Recency
0,189,INDUSTRIAL,10/1/2021,9_W5_2021,S154,สาขาธาตุพนม,Swine,1000005614,FR PK BTG หนังหมู A-L 10KG,32500.0,119
1,189,INDUSTRIAL,10/1/2021,9_W5_2021,S154,สาขาธาตุพนม,Swine,1000005609,FR PK BTG หนังหมู A-B 10KG,12720.0,113
2,189,INDUSTRIAL,10/1/2021,9_W5_2021,S154,สาขาธาตุพนม,Swine,1000005616,FR PK BTG หนังหมู A-S 10KG,9000.0,113
3,630,FOOD SERVICE,10/1/2021,9_W5_2021,S035,สาขามุกดาหาร,Swine,1000004760,FR PK BTG สันคอหมู B-4 FZ,2608.8,113
4,277,FOOD SERVICE,10/1/2021,9_W5_2021,S154,สาขาธาตุพนม,Swine,1000005503,FR PK BTG สันนอกหมูสไลด์ 4mm 1KG/K,2300.0,119
...,...,...,...,...,...,...,...,...,...,...,...
31406,408,WHOLESALE,1/31/2022,2_W1_2022,S154,สาขาธาตุพนม,Egg,1000015235,Egg No.3 (Betagro) Pack 150,1305.0,1
31407,121,WHOLESALE,1/31/2022,2_W1_2022,S154,สาขาธาตุพนม,Egg,1000015238,EGG NO.4 (BETAGRO) PACK 150,1215.0,1
31408,193,WHOLESALE,1/31/2022,2_W1_2022,S154,สาขาธาตุพนม,Egg,1000015238,EGG NO.4 (BETAGRO) PACK 150,1215.0,1
31409,402,WHOLESALE,1/31/2022,2_W1_2022,S031,สาขาหนองคาย,Egg,1000005937,Egg X-Tra Large (Betagro) Pack 10,792.0,1


In [11]:
df['Postdate'] = pd.to_datetime(df['CalendarDay'])

In [12]:
# df.isnull().mean()

**We have all the crucial information we need:**
- Customer ID
- Unit Price
- Quantity
- Invoice Date

**Aggregating the Orders by Month**

In [13]:
# df_agg= df.groupby("Postdate").Recency.sum()
# df_agg

In [14]:
# #converting series to dataframe and resetting index, and changing the column name to 'Orders'
# df_agg=pd.DataFrame(df_agg)
# df_agg=df_agg.reset_index()
# df_agg.head()

In [15]:

# def plot_df(df, x, y, title="", xlabel='Date', ylabel='Orders', dpi=100):
#     plt.figure(figsize=(16,5), dpi=dpi)
#     plt.gca().set(title=title, xlabel=xlabel, ylabel=ylabel)
#     plt.plot(x, y, color='tab:Blue', marker='o')
#     plt.show()

In [16]:
# plot_df(df_agg, x=df_agg.Postdate, y=df_agg.Recency,title='Orders in 2011')

### Calculating  Revenue

**Revenue = Order Count * Average Revenue per Order**

In [17]:
# df['Revenue'] = df['Quantity']*df['UnitPrice']

In [18]:
# sns.boxplot(x=df['Quantity'])

In [19]:
# df.head()

### Monthly Revenue

In [20]:
# df_revenue = df.groupby(['InvoiceYearMonth'])['Revenue'].sum().reset_index()
# df_revenue.tail()

In [21]:
# plot_data = [
#     go.Scatter(
#         x=df_revenue['InvoiceYearMonth'],
#         y=df_revenue['Revenue'],
#         mode='lines+markers'
#     )
# ]

# plot_layout = go.Layout(
#         xaxis={"type": "category"},
#         title='Montly Revenue'
#     )
# fig = go.Figure(data=plot_data, layout=plot_layout)
# pyoff.iplot(fig)

## RFM Analysis

<b>Recency-Frequency-Monetary
    
    
RFM analysis allows you to segment customers by the frequency and value of purchases and identify those customers who spend the most money.

* Recency — how long it’s been since a customer bought something from us
* Frequency — how often a customer buys from us
* Monetary value — the total value of purchases a customer has made</b>

In [22]:
NOW = dt.date(2022,2,1) 
df['Date'] = pd.DatetimeIndex(df.Postdate).date

#### Recency

In [23]:
df_recency = df.groupby(['รหัสลูกค้า'],as_index=False)['Date'].max()
df_recency.columns = ['รหัสลูกค้า','Last_Purchase_Date']

df_recency['Recency'] = df_recency.Last_Purchase_Date.apply(lambda x:(NOW - x).days)
df_recency.drop(columns=['Last_Purchase_Date'],inplace=True)

#### Frequency - Monetarty

In [24]:
FM_Table = df.groupby('รหัสลูกค้า').agg({'รหัสลูกค้า': lambda x:len(x),'Sales_Excld'  : lambda x:x.sum()})
FM_Table.rename(columns = {'รหัสลูกค้า' :'Frequency','Sales_Excld':'Monetary'},inplace= True)

In [25]:
FM_Table

,Frequency,Monetary
รหัสลูกค้า,,
1,104,77560.83
2,85,121976.60
3,34,31417.00
4,128,950734.03
5,465,313464.16
...,...,...
705,4,6278.40
706,1,1209.60
707,6,4050.37


In [26]:
RFM_Table = df_recency.merge(FM_Table,left_on='รหัสลูกค้า',right_on='รหัสลูกค้า')
RFM_Table['FM_Avg'] = (RFM_Table['Frequency'] * RFM_Table['Monetary'])/2
RFM_Table.head(10)

,รหัสลูกค้า,Recency,Frequency,Monetary,FM_Avg
0,1,1,104,77560.83,4.033163e+06
1,2,1,85,121976.60,5.184006e+06
2,3,8,34,31417.00,5.340890e+05
3,4,1,128,950734.03,6.084698e+07
4,5,1,465,313464.16,7.288042e+07
5,6,48,3,1914.00,2.871000e+03
6,7,14,20,3338.51,3.338510e+04
7,8,33,7,4472.87,1.565505e+04
8,9,33,68,118101.53,4.015452e+06
9,10,4,70,383364.00,1.341774e+07


In [27]:
# (NOW - dt.date(2021,2,1)).days == 365

<b>Now we split the metrics into segments using quantiles. 
We will assign a score from 1 to 5 to each Recency, Frequency and Monetary respectively. 5 is the highest value, and 1 is the lowest value. A final RFM score (Overall Value) is calculated simply by combining individual RFM score numbers.</b>

In [28]:
quantiles = RFM_Table.quantile(q=[0.2, 0.4, 0.6, 0.8])
quantiles = quantiles.to_dict()

In [29]:
segmented_rfm = RFM_Table.copy()

In [30]:
def RScore(x,p,d):
    if x <= d[p][0.2]:
        return 5
    elif x <= d[p][0.4]:
        return 4
    elif x <= d[p][0.6]: 
        return 3
    elif x <= d[p][0.8]: 
        return 2
    else:
        return 1
    
def FMScore(x,p,d):
    if x <= d[p][0.2]:
        return 1
    elif x <= d[p][0.4]:
        return 2
    elif x <= d[p][0.6]: 
        return 3
    elif x <= d[p][0.8]: 
        return 4
    else:
        return 5


In [31]:
segmented_rfm['R_Score'] = segmented_rfm['Recency'].apply(RScore, args=('Recency',quantiles))
segmented_rfm['F_Score'] = segmented_rfm['Frequency'].apply(FMScore, args=('Frequency',quantiles))
segmented_rfm['M_Score'] = segmented_rfm['Monetary'].apply(FMScore, args=('Monetary',quantiles))
segmented_rfm['FM_Score'] = segmented_rfm['FM_Avg'].apply(FMScore, args=('FM_Avg',quantiles))
segmented_rfm['RFM_Segment'] = segmented_rfm.R_Score.map(str)+segmented_rfm.F_Score.map(str)+segmented_rfm.M_Score.map(str)
segmented_rfm['RFM_Score'] = segmented_rfm[['R_Score','F_Score','M_Score']].sum(axis=1)


segmented_rfm.head(10)



,รหัสลูกค้า,Recency,Frequency,Monetary,FM_Avg,R_Score,F_Score,M_Score,FM_Score,RFM_Segment,RFM_Score
0,1,1,104,77560.83,4.033163e+06,5,5,4,5,554,14
1,2,1,85,121976.60,5.184006e+06,5,5,5,5,555,15
2,3,8,34,31417.00,5.340890e+05,3,4,4,4,344,11
3,4,1,128,950734.03,6.084698e+07,5,5,5,5,555,15
4,5,1,465,313464.16,7.288042e+07,5,5,5,5,555,15
5,6,48,3,1914.00,2.871000e+03,2,1,2,2,212,5
6,7,14,20,3338.51,3.338510e+04,3,3,2,3,332,8
7,8,33,7,4472.87,1.565505e+04,2,2,2,2,222,6
8,9,33,68,118101.53,4.015452e+06,2,4,5,5,245,11
9,10,4,70,383364.00,1.341774e+07,4,4,5,5,445,13


**Calculating the total RFM score combined**

RFM_Score = R_quartile + F_quartile + M_quartile

In [32]:
segmented_rfm['RFM_Score'].unique()
segmented_rfm.to_excel("segmented_rfm_1.xlsx")
segmented_rfm

,รหัสลูกค้า,Recency,Frequency,Monetary,FM_Avg,R_Score,F_Score,M_Score,FM_Score,RFM_Segment,RFM_Score
0,1,1,104,77560.83,4033163.16,5,5,4,5,554,14
1,2,1,85,121976.60,5184005.50,5,5,5,5,555,15
2,3,8,34,31417.00,534089.00,3,4,4,4,344,11
3,4,1,128,950734.03,60846977.92,5,5,5,5,555,15
4,5,1,465,313464.16,72880417.20,5,5,5,5,555,15
...,...,...,...,...,...,...,...,...,...,...,...
704,705,5,4,6278.40,12556.80,4,2,2,2,422,8
705,706,6,1,1209.60,604.80,3,1,1,1,311,5
706,707,3,6,4050.37,12151.11,4,2,2,2,422,8
707,708,7,22,5063.80,55701.80,3,3,2,3,332,8


## 2D segmentation

In [33]:
segmented_RFXM = segmented_rfm.copy()
segmented_RFXM

,รหัสลูกค้า,Recency,Frequency,Monetary,FM_Avg,R_Score,F_Score,M_Score,FM_Score,RFM_Segment,RFM_Score
0,1,1,104,77560.83,4033163.16,5,5,4,5,554,14
1,2,1,85,121976.60,5184005.50,5,5,5,5,555,15
2,3,8,34,31417.00,534089.00,3,4,4,4,344,11
3,4,1,128,950734.03,60846977.92,5,5,5,5,555,15
4,5,1,465,313464.16,72880417.20,5,5,5,5,555,15
...,...,...,...,...,...,...,...,...,...,...,...
704,705,5,4,6278.40,12556.80,4,2,2,2,422,8
705,706,6,1,1209.60,604.80,3,1,1,1,311,5
706,707,3,6,4050.37,12151.11,4,2,2,2,422,8
707,708,7,22,5063.80,55701.80,3,3,2,3,332,8


In [52]:
segmented_RFXM.loc[(segmented_RFXM['R_Score'] == 1) & (segmented_RFXM['FM_Score'] == 1), 'Class'] = 'Lost'
segmented_RFXM.loc[(segmented_RFXM['R_Score'] >= 1) & (segmented_RFXM['R_Score'] <= 2) & (segmented_RFXM['FM_Score'] >= 1) & (segmented_RFXM['FM_Score'] <= 2), 'Class'] = 'Hibernating'
segmented_RFXM.loc[(segmented_RFXM['R_Score'] == 1) & (segmented_RFXM['FM_Score'] <= 2) & (segmented_RFXM['FM_Score'] <= 5), 'Class'] = "Can't lose them"
segmented_RFXM.loc[(segmented_RFXM['R_Score'] >= 1) & (segmented_RFXM['R_Score'] <= 2) & (segmented_RFXM['FM_Score'] >= 2) & (segmented_RFXM['FM_Score'] <= 5), 'Class'] = "At Risk"
segmented_RFXM.loc[(segmented_RFXM['R_Score'] >= 2) & (segmented_RFXM['R_Score'] <= 3) & (segmented_RFXM['FM_Score'] <= 2), 'Class'] = 'About To Sleep'
segmented_RFXM.loc[(segmented_RFXM['R_Score'] >= 2) & (segmented_RFXM['R_Score'] <= 3) & (segmented_RFXM['FM_Score'] >= 2) & (segmented_RFXM['FM_Score'] <= 3), 'Class'] = 'Need Attention'
segmented_RFXM.loc[(segmented_RFXM['R_Score'] >= 3) & (segmented_RFXM['R_Score'] <= 4) & (segmented_RFXM['FM_Score'] == 1), 'Class'] = 'Promising'
segmented_RFXM.loc[(segmented_RFXM['R_Score'] >= 4) & (segmented_RFXM['R_Score'] <= 5) & (segmented_RFXM['FM_Score'] == 1), 'Class'] = 'New Customer'
segmented_RFXM.loc[(segmented_RFXM['R_Score'] >= 3) & (segmented_RFXM['R_Score'] <= 5) & (segmented_RFXM['FM_Score'] >= 1) & (segmented_RFXM['FM_Score'] <= 3), 'Class'] = 'Potential Loyal'
segmented_RFXM.loc[(segmented_RFXM['R_Score'] >= 2) & (segmented_RFXM['R_Score'] <= 5) & (segmented_RFXM['FM_Score'] >= 3) & (segmented_RFXM['FM_Score'] <= 5), 'Class'] = 'Loyal'
segmented_RFXM.loc[(segmented_RFXM['R_Score'] >= 4) & (segmented_RFXM['R_Score'] <= 5) & (segmented_RFXM['FM_Score'] >= 4) & (segmented_RFXM['FM_Score'] <= 5), 'Class'] = 'Champion'

# Champion
# Loyal
# PoLoyal 
# NewCust
# Promis
# NeedAttent
# About
# risk
# Cantlose
# Hibernate
# Lostsegmented_RFXMsegmented_RFXM

RFXMChampion = segmented_RFXM[segmented_RFXM['Class'] == 'Champion']
RFXMLoyal = segmented_RFXM[segmented_RFXM['Class'] == 'Loyal']
RFXMPoLoyal = segmented_RFXM[segmented_RFXM['Class'] == 'Potential Loyal']
RFXMNewCust = segmented_RFXM[segmented_RFXM['Class'] == 'New Customer']
RFXMPromis = segmented_RFXM[segmented_RFXM['Class'] == 'Promising']
RFXMNeedAttent = segmented_RFXM[segmented_RFXM['Class'] == 'Need Attention']
RFXMAbout = segmented_RFXM[segmented_RFXM['Class'] == 'About to sleep']
RFXMrisk = segmented_RFXM[segmented_RFXM['Class'] == 'At risk']
RFXMCantlose = segmented_RFXM[segmented_RFXM['Class'] == "Can't lose them"]
RFXMHibernate = segmented_RFXM[segmented_RFXM['Class'] == 'Hibernating']
RFXMLost = segmented_RFXM[segmented_RFXM['Class'] == 'Lost']

print('Champion: ', len(segmented_RFXM[segmented_RFXM['Class'] == 'Champion']))
print('Loyal: ', len(segmented_RFXM[segmented_RFXM['Class'] == 'Loyal']))
print('Potential Loyal: ', len(segmented_RFXM[segmented_RFXM['Class'] == 'Potential Loyal']))
print('New Customer: ', len(segmented_RFXM[segmented_RFXM['Class'] == 'New Customer']))
print('Promising: ', len(segmented_RFXM[segmented_RFXM['Class'] == 'Promising']))
print('Need Attention: ', len(segmented_RFXM[segmented_RFXM['Class'] == 'Need Attention']))
print('About to sleep: ', len(segmented_RFXM[segmented_RFXM['Class'] == 'About to sleep']))
print('At risk: ', len(segmented_RFXM[segmented_RFXM['Class'] == 'At risk']))
print("Can't lose Them: ", len(segmented_RFXM[segmented_RFXM['Class'] == "Can't lose them"]))
print('Hibernating: ', len(segmented_RFXM[segmented_RFXM['Class'] == 'Hibernating']))
print('Lost: ', len(segmented_RFXM[segmented_RFXM['Class'] == 'Lost']))



Champion:  208
Loyal:  197
Potential Loyal:  70
New Customer:  0
Promising:  0
Need Attention:  47
About to sleep:  0
At risk:  0
Can't lose Them:  75
Hibernating:  0
Lost:  0


In [53]:
segmented_RFXM['RFM_Score'].unique()
segmented_RFXM.to_excel("segmented_rfm_1.xlsx")
segmented_RFXM

,รหัสลูกค้า,Recency,Frequency,Monetary,FM_Avg,R_Score,F_Score,M_Score,FM_Score,RFM_Segment,RFM_Score,Class
0,1,1,104,77560.83,4033163.16,5,5,4,5,554,14,Champion
1,2,1,85,121976.60,5184005.50,5,5,5,5,555,15,Champion
2,3,8,34,31417.00,534089.00,3,4,4,4,344,11,Loyal
3,4,1,128,950734.03,60846977.92,5,5,5,5,555,15,Champion
4,5,1,465,313464.16,72880417.20,5,5,5,5,555,15,Champion
...,...,...,...,...,...,...,...,...,...,...,...,...
704,705,5,4,6278.40,12556.80,4,2,2,2,422,8,Potential Loyal
705,706,6,1,1209.60,604.80,3,1,1,1,311,5,Potential Loyal
706,707,3,6,4050.37,12151.11,4,2,2,2,422,8,Potential Loyal
707,708,7,22,5063.80,55701.80,3,3,2,3,332,8,Loyal


In [54]:
# Champion
# Loyal
# Promis
# NewCust
# Warm
# Cold
# NeedAttent
# Cantlose
# Sleepers
# Lost

xChampion = RFXMChampion.Recency
xLoyal = RFXMLoyal.Recency
xPoLoyal = RFXMPoLoyal.Recency
xNewCust = RFXMNewCust.Recency
xPromis = RFXMPromis.Recency
xNeedAttent = RFXMNeedAttent.Recency
xAbout = RFXMAbout.Recency
xrisk = RFXMrisk.Recency
xCantlose = RFXMCantlose.Recency
xHibernate = RFXMHibernate.Recency
xLost = RFXMLost.Recency

yChampion = RFXMChampion.FM_Avg
yLoyal = RFXMLoyal.FM_Avg
yPoLoyal = RFXMPoLoyal.FM_Avg
yNewCust = RFXMNewCust.FM_Avg
yPromis = RFXMPromis.FM_Avg
yNeedAttent = RFXMNeedAttent.FM_Avg
yAbout = RFXMAbout.FM_Avg
yrisk = RFXMrisk.FM_Avg
yCantlose = RFXMCantlose.FM_Avg
yHibernate = RFXMHibernate.FM_Avg
yLost = RFXMLost.FM_Avg

logyChampion = np.log(RFXMChampion.FM_Avg)
logyLoyal = np.log(RFXMLoyal.FM_Avg)
logyPoLoyal = np.log(RFXMPoLoyal.FM_Avg)
logyNewCust = np.log(RFXMNewCust.FM_Avg)
logyPromis = np.log(RFXMPromis.FM_Avg)
logyNeedAttent = np.log(RFXMNeedAttent.FM_Avg)
logyAbout = np.log(RFXMAbout.FM_Avg)
logyrisk = np.log(RFXMrisk.FM_Avg)
logyCantlose = np.log(RFXMCantlose.FM_Avg)
logyHibernate = np.log(RFXMHibernate.FM_Avg)
logyLost = np.log(RFXMLost.FM_Avg)




In [55]:
fig = plt.figure(figsize=(10, 10))
ax = plt.axes()

plt.title('Data Visualization')
ax.set_ylabel('Frequency * Monetary')
ax.set_xlabel('Recency')


ax.scatter(xChampion, yChampion, c='#23237C', linewidth=2);
ax.scatter(xLoyal, yLoyal, c='#6061C9', linewidth=2);
ax.scatter(xPoLoyal, yPoLoyal, c='#B5B8F9', linewidth=2);
ax.scatter(xNewCust, yNewCust, c='#ABEEEC', linewidth=2);
ax.scatter(xPromis, yPromis, c='#01C4C3', linewidth=2);
ax.scatter(xNeedAttent, yNeedAttent, c='#B8F080', linewidth=2);
ax.scatter(xAbout, yAbout, c='#FBC57E', linewidth=2);
ax.scatter(xrisk, yrisk, c='#F4C500', linewidth=2);
ax.scatter(xCantlose, yCantlose, c='#EBC9EE', linewidth=2);
ax.scatter(xHibernate, yHibernate, c='#FFEE7C', linewidth=2);
ax.scatter(xLost, yLost, c='#FF9014', linewidth=2);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [56]:
fig = plt.figure(figsize=(10, 10))
ax = plt.axes()

plt.title('Data Visualization')
ax.set_ylabel('Frequency * Monetary')
ax.set_xlabel('Recency')

ax.scatter(xChampion, logyChampion, c='#23237C', linewidth=2);
ax.scatter(xLoyal, logyLoyal, c='#6061C9', linewidth=2);
ax.scatter(xPoLoyal, logyPoLoyal, c='#B5B8F9', linewidth=2);
ax.scatter(xNewCust, logyNewCust, c='#ABEEEC', linewidth=2);
ax.scatter(xPromis, logyPromis, c='#01C4C3', linewidth=2);
ax.scatter(xNeedAttent, logyNeedAttent, c='#B8F080', linewidth=2);
ax.scatter(xAbout, logyAbout, c='#FBC57E', linewidth=2);
ax.scatter(xrisk, logyrisk, c='#F4C500', linewidth=2);
ax.scatter(xCantlose, logyCantlose, c='#EBC9EE', linewidth=2);
ax.scatter(xHibernate, logyHibernate, c='#FFEE7C', linewidth=2);
ax.scatter(xLost, logyLost, c='#FF9014', linewidth=2);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [57]:
segmented_rfm['RFM_Score'].unique()
# segmented_rfm.to_excel("segmented_rfm_1.xlsx")
segmented_rfm

,รหัสลูกค้า,Recency,Frequency,Monetary,FM_Avg,R_Score,F_Score,M_Score,FM_Score,RFM_Segment,RFM_Score,Class
0,1,1,104,77560.83,4033163.16,5,5,4,5,554,14,Champion
1,2,1,85,121976.60,5184005.50,5,5,5,5,555,15,Champion
2,3,8,34,31417.00,534089.00,3,4,4,4,344,11,Loyal
3,4,1,128,950734.03,60846977.92,5,5,5,5,555,15,Champion
4,5,1,465,313464.16,72880417.20,5,5,5,5,555,15,Champion
...,...,...,...,...,...,...,...,...,...,...,...,...
704,705,5,4,6278.40,12556.80,4,2,2,2,422,8,New Customer
705,706,6,1,1209.60,604.80,3,1,1,1,311,5,New Customer
706,707,3,6,4050.37,12151.11,4,2,2,2,422,8,New Customer
707,708,7,22,5063.80,55701.80,3,3,2,3,332,8,Hibernating


## 3D segmentation

In [58]:
# Champion
# Loyal
# PoLoyal 
# NewCust
# Promis
# NeedAttent
# About
# risk
# Cantlose
# Hibernate
# Lost
RBChamp = ['555', '554', '544', '545', '454', '455', '445']

RBLoyal = ['543', '444', '435', '355', '354', '345', '344', '335']

RBPoLoyal = ['553', '551', '552', '541', '542', '533', '532', '531', '452',
             '451', '442', '441', '431', '453', '433', '432', '423', '353',
             '352', '351', '342', '341', '333', '323']

RBNewCust = ['512', '511', '422', '421', '412', '411', '311']

RBPromis = ['525', '524', '523', '522', '521', '515', '514', '513', '425',
            '424', '413', '414', '415', '315', '314', '313']

RBNeedAttent = ['535', '534', '443', '434', '343', '334', '325', '324']

RBAbout = ['331', '321', '312', '221', '213', '231', '241', '251']

RBrisk = ['255', '254', '245', '244', '253', '252', '243', '242', '235', '234',
          '225', '224', '153', '152', '145', '143', '142', '135', '134', '133',
          '125', '124']

RBCantlose = ['155', '154', '144', '214', '215', '115', '114', '113']

RBHibernate = ['332', '322', '233', '232', '223', '222', '132', '123', '122', '212', '211']
RBLost = ['111', '112', '121', '131', '141', '151']


In [59]:
segmented_rfm['Class'] = np.where((segmented_rfm.RFM_Segment.isin(RBChamp)), 'Champion',
                                  np.where((segmented_rfm.RFM_Segment.isin(RBLoyal)), 'Loyal',
                                           np.where((segmented_rfm.RFM_Segment.isin(RBPoLoyal)), 'Potential Loyal',
                                                    np.where((segmented_rfm.RFM_Segment.isin(RBNewCust)), 'New Customer',
                                                             np.where((segmented_rfm.RFM_Segment.isin(RBPromis)), 'Promising',
                                                                      np.where((segmented_rfm.RFM_Segment.isin(RBNeedAttent)), 'Need Attention',
                                                                               np.where((segmented_rfm.RFM_Segment.isin(RBAbout)), 'About to sleep',
                                                                                        np.where((segmented_rfm.RFM_Segment.isin(RBrisk)), 'At risk',
                                                                                                 np.where((segmented_rfm.RFM_Segment.isin(RBCantlose)), "Can't lose them",
                                                                                                          np.where((segmented_rfm.RFM_Segment.isin(RBHibernate)), 'Hibernating',
                                                                                                                   np.where((segmented_rfm.RFM_Segment.isin(RBLost)), 'Lost', 0)))))))))))



Champion = segmented_rfm[segmented_rfm['Class'] == 'Champion']
Loyal = segmented_rfm[segmented_rfm['Class'] == 'Loyal']
PoLoyal = segmented_rfm[segmented_rfm['Class'] == 'Potential Loyal']
NewCust = segmented_rfm[segmented_rfm['Class'] == 'New Customer']
Promis = segmented_rfm[segmented_rfm['Class'] == 'Promising']
NeedAttent = segmented_rfm[segmented_rfm['Class'] == 'Need Attention']
About = segmented_rfm[segmented_rfm['Class'] == 'About to sleep']
risk = segmented_rfm[segmented_rfm['Class'] == 'At risk']
Cantlose = segmented_rfm[segmented_rfm['Class'] == "Can't lose them"]
Hibernate = segmented_rfm[segmented_rfm['Class'] == 'Hibernating']
Lost = segmented_rfm[segmented_rfm['Class'] == 'Lost']
notseg = segmented_rfm[segmented_rfm['Class'] == 0 ]

print('Champion: ', len(segmented_rfm[segmented_rfm['Class'] == 'Champion']))
print('Loyal: ', len(segmented_rfm[segmented_rfm['Class'] == 'Loyal']))
print('Potential Loyal: ', len(segmented_rfm[segmented_rfm['Class'] == 'Potential Loyal']))
print('New Customer: ', len(segmented_rfm[segmented_rfm['Class'] == 'New Customer']))
print('Promising: ', len(segmented_rfm[segmented_rfm['Class'] == 'Promising']))
print('Need Attention: ', len(segmented_rfm[segmented_rfm['Class'] == 'Need Attention']))
print('About to sleep: ', len(segmented_rfm[segmented_rfm['Class'] == 'About to sleep']))
print('At risk: ', len(segmented_rfm[segmented_rfm['Class'] == 'At risk']))
print("Can't lose Them: ", len(segmented_rfm[segmented_rfm['Class'] == "Can't lose them"]))
print('Hibernating: ', len(segmented_rfm[segmented_rfm['Class'] == 'Hibernating']))
print('Lost: ', len(segmented_rfm[segmented_rfm['Class'] == 'Lost']))
print('notseg: ', len(segmented_rfm[segmented_rfm['Class'] == 0]))

segmented_rfm

Champion:  156
Loyal:  70
Potential Loyal:  66
New Customer:  23
Promising:  10
Need Attention:  66
About to sleep:  32
At risk:  37
Can't lose Them:  11
Hibernating:  148
Lost:  90
notseg:  0


,รหัสลูกค้า,Recency,Frequency,Monetary,FM_Avg,R_Score,F_Score,M_Score,FM_Score,RFM_Segment,RFM_Score,Class
0,1,1,104,77560.83,4033163.16,5,5,4,5,554,14,Champion
1,2,1,85,121976.60,5184005.50,5,5,5,5,555,15,Champion
2,3,8,34,31417.00,534089.00,3,4,4,4,344,11,Loyal
3,4,1,128,950734.03,60846977.92,5,5,5,5,555,15,Champion
4,5,1,465,313464.16,72880417.20,5,5,5,5,555,15,Champion
...,...,...,...,...,...,...,...,...,...,...,...,...
704,705,5,4,6278.40,12556.80,4,2,2,2,422,8,New Customer
705,706,6,1,1209.60,604.80,3,1,1,1,311,5,New Customer
706,707,3,6,4050.37,12151.11,4,2,2,2,422,8,New Customer
707,708,7,22,5063.80,55701.80,3,3,2,3,332,8,Hibernating


In [60]:
# Champion
# Loyal
# PoLoyal 
# NewCust
# Promis
# NeedAttent
# About
# risk
# Cantlose
# Hibernate
# Lost

xChampion = Champion.Recency
xLoyal = Loyal.Recency
xPoLoyal = PoLoyal.Recency
xNewCust = NewCust.Recency
xPromis = Promis.Recency
xNeedAttent = NeedAttent.Recency
xAbout = About.Recency
xrisk = risk.Recency
xCantlose = Cantlose.Recency
xHibernate = Hibernate.Recency
xLost = Lost.Recency

yChampion = Champion.Frequency
yLoyal = Loyal.Frequency
yPoLoyal = PoLoyal.Frequency
yNewCust = NewCust.Frequency
yPromis = Promis.Frequency
yNeedAttent = NeedAttent.Frequency
yAbout = About.Frequency
yrisk = risk.Frequency
yCantlose = Cantlose.Frequency
yHibernate = Hibernate.Frequency
yLost = Lost.Frequency

zChampion = Champion.Monetary
zLoyal = Loyal.Monetary
zPoLoyal = PoLoyal.Monetary
zNewCust = NewCust.Monetary
zPromis = Promis.Monetary
zNeedAttent = NeedAttent.Monetary
zAbout = About.Monetary
zrisk = risk.Monetary
zCantlose = Cantlose.Monetary
zHibernate = Hibernate.Monetary
zLost = Lost.Monetary

logyChampion = np.log(Champion.Frequency)
logyLoyal = np.log(Loyal.Frequency)
logyPoLoyal = np.log(PoLoyal.Frequency)
logyNewCust = np.log(NewCust.Frequency)
logyPromis = np.log(Promis.Frequency)
logyNeedAttent = np.log(NeedAttent.Frequency)
logyAbout = np.log(About.Frequency)
logyrisk = np.log(risk.Frequency)
logyCantlose = np.log(Cantlose.Frequency)
logyHibernate = np.log(Hibernate.Frequency)
logyLost = np.log(Lost.Frequency)

logzChampion = np.log(Champion.Monetary)
logzLoyal = np.log(Loyal.Monetary)
logzPoLoyal = np.log(PoLoyal.Monetary)
logzNewCust = np.log(NewCust.Monetary)
logzPromis = np.log(Promis.Monetary)
logzNeedAttent = np.log(NeedAttent.Monetary)
logzAbout = np.log(About.Monetary)
logzrisk = np.log(risk.Monetary)
logzCantlose = np.log(Cantlose.Monetary)
logzHibernate = np.log(Hibernate.Monetary)
logzLost = np.log(Lost.Monetary)


In [61]:
fig = plt.figure(figsize=(10, 10))
ax = plt.axes(projection='3d')
plt.title('Data Visualization')
ax.set_xlabel('Recency')
ax.set_ylabel('Frequency')
ax.set_zlabel('Monetary')

ax.scatter(xChampion, yChampion, zChampion, c='#23237C', linewidth=2);
ax.scatter(xLoyal, yLoyal, zLoyal, c='#6061C9', linewidth=2);
ax.scatter(xPoLoyal, yPoLoyal, zPoLoyal, c='#B5B8F9', linewidth=2);
ax.scatter(xNewCust, yNewCust, zNewCust, c='#ABEEEC', linewidth=2);
ax.scatter(xPromis, yPromis, zPromis, c='#01C4C3', linewidth=2);
ax.scatter(xNeedAttent, yNeedAttent, zNeedAttent, c='#B8F080', linewidth=2);
ax.scatter(xAbout, yAbout, zAbout, c='#FBC57E', linewidth=2);
ax.scatter(xrisk, yrisk, zrisk, c='#F4C500', linewidth=2);
ax.scatter(xCantlose, yCantlose, zCantlose, c='#EBC9EE', linewidth=2);
ax.scatter(xHibernate, yHibernate, zHibernate, c='#FFEE7C', linewidth=2);
ax.scatter(xLost, yLost, zLost, c='#FF9014', linewidth=2);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [62]:

fig = plt.figure(figsize=(10, 10))
ax = plt.axes(projection='3d')

plt.title('Data Visualization')
ax.set_xlabel('Recency')
ax.set_ylabel('Log Frequency')
ax.set_zlabel('Log Monetary')

ax.scatter(xChampion, logyChampion, logzChampion, c='#23237C', linewidth=2);
ax.scatter(xLoyal, logyLoyal, logzLoyal, c='#6061C9', linewidth=2);
ax.scatter(xPoLoyal, logyPoLoyal, logzPoLoyal, c='#B5B8F9', linewidth=2);
ax.scatter(xNewCust, logyNewCust, logzNewCust, c='#ABEEEC', linewidth=2);
ax.scatter(xPromis, logyPromis, logzPromis, c='#01C4C3', linewidth=2);
ax.scatter(xNeedAttent, logyNeedAttent, logzNeedAttent, c='#B8F080', linewidth=2);
ax.scatter(xAbout, logyAbout, logzAbout, c='#FBC57E', linewidth=2);
ax.scatter(xrisk, logyrisk, logzrisk, c='#F4C500', linewidth=2);
ax.scatter(xCantlose, logyCantlose, logzCantlose, c='#EBC9EE', linewidth=2);
ax.scatter(xHibernate, logyHibernate, logzHibernate, c='#FFEE7C', linewidth=2);
ax.scatter(xLost, logyLost, logzLost, c='#FF9014', linewidth=2);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [63]:
fig = plt.figure(figsize=(10, 10))
ax = plt.axes()

plt.title('Data Visualization')
ax.set_ylabel('Frequency')
ax.set_xlabel('Recency')


ax.scatter(xChampion, yChampion,  c='#23237C', linewidth=2);
ax.scatter(xLoyal, yLoyal,  c='#6061C9', linewidth=2);
ax.scatter(xPoLoyal, yPoLoyal,  c='#B5B8F9', linewidth=2);
ax.scatter(xNewCust, yNewCust,  c='#ABEEEC', linewidth=2);
ax.scatter(xPromis, yPromis,  c='#01C4C3', linewidth=2);
ax.scatter(xNeedAttent, yNeedAttent,  c='#B8F080', linewidth=2);
ax.scatter(xAbout, yAbout,  c='#FBC57E', linewidth=2);
ax.scatter(xrisk, yrisk,  c='#F4C500', linewidth=2);
ax.scatter(xCantlose, yCantlose,  c='#EBC9EE', linewidth=2);
ax.scatter(xHibernate, yHibernate,  c='#FFEE7C', linewidth=2);
ax.scatter(xLost, yLost,  c='#FF9014', linewidth=2);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [64]:
fig = plt.figure(figsize=(10, 10))
ax = plt.axes()


plt.title('Data Visualization')
ax.set_ylabel('Log Frequency')
ax.set_xlabel('Recency')


ax.scatter(xChampion, logyChampion,  c='#23237C', linewidth=2);
ax.scatter(xLoyal, logyLoyal,  c='#6061C9', linewidth=2);
ax.scatter(xPoLoyal, logyPoLoyal,  c='#B5B8F9', linewidth=2);
ax.scatter(xNewCust, logyNewCust,  c='#ABEEEC', linewidth=2);
ax.scatter(xPromis, logyPromis,  c='#01C4C3', linewidth=2);
ax.scatter(xNeedAttent, logyNeedAttent,  c='#B8F080', linewidth=2);
ax.scatter(xAbout, logyAbout,  c='#FBC57E', linewidth=2);
ax.scatter(xrisk, logyrisk,  c='#F4C500', linewidth=2);
ax.scatter(xCantlose, logyCantlose,  c='#EBC9EE', linewidth=2);
ax.scatter(xHibernate, logyHibernate,  c='#FFEE7C', linewidth=2);
ax.scatter(xLost, logyLost,  c='#FF9014', linewidth=2);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [65]:
fig = plt.figure(figsize=(10, 10))
ax = plt.axes()

plt.title('Data Visualization')
ax.set_xlabel('Recency')
ax.set_ylabel('Monetary')

ax.scatter( xChampion, zChampion, c='#23237C', linewidth=2);
ax.scatter( xLoyal, zLoyal, c='#6061C9', linewidth=2);
ax.scatter( xPoLoyal, zPoLoyal, c='#B5B8F9', linewidth=2);
ax.scatter( xNewCust, zNewCust, c='#ABEEEC', linewidth=2);
ax.scatter( xPromis, zPromis, c='#01C4C3', linewidth=2);
ax.scatter( xNeedAttent, zNeedAttent, c='#B8F080', linewidth=2);
ax.scatter( xAbout, zAbout, c='#FBC57E', linewidth=2);
ax.scatter( xrisk, zrisk, c='#F4C500', linewidth=2);
ax.scatter( xCantlose, zCantlose, c='#EBC9EE', linewidth=2);
ax.scatter( xHibernate, zHibernate, c='#FFEE7C', linewidth=2);
ax.scatter( xLost, zLost, c='#FF9014', linewidth=2);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [48]:
fig = plt.figure(figsize=(10, 10))
ax = plt.axes()


plt.title('Data Visualization')
ax.set_xlabel('Recency')
ax.set_ylabel('Log Monetary')

ax.scatter( xChampion, logzChampion, c='#23237C', linewidth=2);
ax.scatter( xLoyal, logzLoyal, c='#6061C9', linewidth=2);
ax.scatter( xPoLoyal, logzPoLoyal, c='#B5B8F9', linewidth=2);
ax.scatter( xNewCust, logzNewCust, c='#ABEEEC', linewidth=2);
ax.scatter( xPromis, logzPromis, c='#01C4C3', linewidth=2);
ax.scatter( xNeedAttent, logzNeedAttent, c='#B8F080', linewidth=2);
ax.scatter( xAbout, logzAbout, c='#FBC57E', linewidth=2);
ax.scatter( xrisk, logzrisk, c='#F4C500', linewidth=2);
ax.scatter( xCantlose, logzCantlose, c='#EBC9EE', linewidth=2);
ax.scatter( xHibernate, logzHibernate, c='#FFEE7C', linewidth=2);
ax.scatter( xLost, logzLost, c='#FF9014', linewidth=2);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [66]:
fig = plt.figure(figsize=(10, 10))
ax = plt.axes()


plt.title('Data Visualization')
ax.set_xlabel('Frequency')
ax.set_ylabel('Monetary')

ax.scatter(yChampion,  zChampion, c='#23237C', linewidth=2);
ax.scatter(yLoyal,  zLoyal, c='#6061C9', linewidth=2);
ax.scatter(yPoLoyal,  zPoLoyal, c='#B5B8F9', linewidth=2);
ax.scatter(yNewCust,  zNewCust, c='#ABEEEC', linewidth=2);
ax.scatter(yPromis,  zPromis, c='#01C4C3', linewidth=2);
ax.scatter(yNeedAttent,  zNeedAttent, c='#B8F080', linewidth=2);
ax.scatter(yAbout,  zAbout, c='#FBC57E', linewidth=2);
ax.scatter(yrisk,  zrisk, c='#F4C500', linewidth=2);
ax.scatter(yCantlose,  zCantlose, c='#EBC9EE', linewidth=2);
ax.scatter(yHibernate,  zHibernate, c='#FFEE7C', linewidth=2);
ax.scatter(yLost,  zLost, c='#FF9014', linewidth=2);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [67]:
fig = plt.figure(figsize=(10, 10))
ax = plt.axes()

plt.title('Data Visualization')
ax.set_xlabel('Log Frequency')
ax.set_ylabel('Log Monetary')

ax.scatter(logyChampion, logzChampion, c='#23237C', linewidth=2);
ax.scatter(logyLoyal, logzLoyal, c='#6061C9', linewidth=2);
ax.scatter(logyPoLoyal, logzPoLoyal, c='#B5B8F9', linewidth=2);
ax.scatter(logyNewCust, logzNewCust, c='#ABEEEC', linewidth=2);
ax.scatter(logyPromis, logzPromis, c='#01C4C3', linewidth=2);
ax.scatter(logyNeedAttent, logzNeedAttent, c='#B8F080', linewidth=2);
ax.scatter(logyAbout, logzAbout, c='#FBC57E', linewidth=2);
ax.scatter(logyrisk, logzrisk, c='#F4C500', linewidth=2);
ax.scatter(logyCantlose, logzCantlose, c='#EBC9EE', linewidth=2);
ax.scatter(logyHibernate, logzHibernate, c='#FFEE7C', linewidth=2);
ax.scatter(logyLost, logzLost, c='#FF9014', linewidth=2);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [51]:
segmented_rfm.groupby('RFM_Score').agg({
    'Recency': 'mean',
    'Frequency': 'mean',
    'Monetary': ['mean', 'count'] }).round(1)

Recency Frequency  Monetary      
             mean      mean      mean count
RFM_Score                                  
3            87.8       1.5     576.1    61
4            57.7       2.3    1111.3    62
5            50.8       3.8    3242.0    65
6            33.8       4.8    7410.2    53
7            28.4      10.6    7823.7    40
8            24.9      12.7   63984.0    44
9            14.6      18.6   29471.0    47
10           10.2      24.6   36516.2    51
11            7.6      35.7   50575.2    69
12            5.0      49.9   93233.2    52
13            3.5      78.9  113247.2    53
14            2.1     100.4  236906.8    41
15            1.0     200.1  380623.0    71

## K-means to perform RFM (Recency, Frequency, and Monetary) customer segmentation. 

<b>K-means gives the best result under the following conditions:</b>
- <b>Data’s distribution is not skewed (i.e. long-tail distribution) </b>
- <b> Data is standardised (i.e. mean of 0 and standard deviation of 1).</b>

**Plotting distribution charts to check for skewness.**

In [42]:
## Function to check skewness
def check_skew(df_skew, column):
    skew = stats.skew(df_skew[column])
    skewtest = stats.skewtest(df_skew[column])
    plt.title('Distribution of ' + column)
    sns.distplot(df_skew[column])
    print("{}'s: Skew: {}, : {}".format(column, skew, skewtest))
    return

In [43]:
# Plot all 3 graphs together for summary findings
plt.figure(figsize=(9, 9))

plt.subplot(3, 1, 1)
check_skew(RFM_Table,'Recency')

plt.subplot(3, 1, 2)
check_skew(RFM_Table,'Frequency')

plt.subplot(3, 1, 3)
check_skew(RFM_Table,'Monetary')
plt.show()
# plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Recency's: Skew: 1.2964679755700776, : SkewtestResult(statistic=12.138730210032515, pvalue=6.583983228077953e-34)
Frequency's: Skew: 4.873802454537846, : SkewtestResult(statistic=24.38603617960593, pvalue=2.4055782312886885e-131)
Monetary's: Skew: 5.812749523514164, : SkewtestResult(statistic=26.10220820539456, pvalue=3.441315119453498e-150)


<b>The data is highly skewed,therefore we will perform log transformations to reduce the skewness of each variable. I added a small constant as log transformation demands all the values to be positive.</b> 

In [44]:
# cols=['Recency','Frequency','Monetary_Value']
# df_rfm_log = RFM_Table[cols]

In [45]:
df_rfm_log = RFM_Table.copy()

In [46]:
df_rfm_log.head()

,CustomerID,Recency,Frequency,Monetary
0,100000004,33,36,2735900.0
1,100000066,11,8,50152.0
2,100000105,113,10,501930.0
3,100000130,28,7,44760.0
4,100000149,222,1,478720.0


In [47]:
df_rfm_log = np.log(df_rfm_log+1)

plt.figure(figsize=(9, 9))

plt.subplot(3, 1, 1)
check_skew(df_rfm_log,'Recency')

plt.subplot(3, 1, 2)
check_skew(df_rfm_log,'Frequency')
plt.subplot(3, 1, 3)
check_skew(df_rfm_log,'Monetary')

# plt.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Recency's: Skew: -0.059884620630825086, : SkewtestResult(statistic=-0.712882096256282, pvalue=0.4759187188430598)
Frequency's: Skew: 0.5412534672224372, : SkewtestResult(statistic=6.059951762575587, pvalue=1.3616237669595843e-09)
Monetary's: Skew: -0.015226198174268334, : SkewtestResult(statistic=-0.18140438581333618, pvalue=0.8560501766105817)


In [48]:
RFM_Table.describe()

,CustomerID,Recency,Frequency,Monetary
count,8.370000e+02,837.000000,837.000000,8.370000e+02
mean,1.000144e+08,89.102748,7.309438,4.398213e+05
std,1.110386e+04,89.679541,9.464038,1.030854e+06
min,1.000000e+08,5.000000,1.000000,3.600000e+02
25%,1.000119e+08,18.000000,2.000000,2.400000e+04
50%,1.000134e+08,56.000000,5.000000,1.083000e+05
75%,1.000148e+08,126.000000,9.000000,3.632500e+05
max,1.000510e+08,363.000000,126.000000,1.382913e+07


In [49]:
windsoriser = Winsorizer(tail='both', # cap left, right or both tails 
                          fold=2,
                           variables=[ 'Recency', 'Frequency', 'Monetary']
                        )
windsoriser.fit(df_rfm_log)

Winsorizer(fold=2, tail='both', variables=['Recency', 'Frequency', 'Monetary'])

In [50]:
df_rfm_log = windsoriser.transform(df_rfm_log)

**Once the skewness is reduced, I standardised the data by centring and scaling. Note all the variables now have a mean of 0 and a standard deviation of 1.**

In [51]:
scaler = StandardScaler()

scaler.fit(df_rfm_log)

RFM_Table_scaled = scaler.transform(df_rfm_log)

In [52]:
RFM_Table_scaled = pd.DataFrame(RFM_Table_scaled, columns=df_rfm_log.columns)
RFM_Table_scaled.head()

,CustomerID,Recency,Frequency,Monetary
0,-1.299426,-0.379115,2.113771,1.851283
1,-1.293838,-1.311533,0.592184,-0.379721
2,-1.290323,0.704058,0.859532,0.905271
3,-1.288069,-0.521526,0.435264,-0.443173
4,-1.286357,1.304783,-1.411660,0.878859


## K-Means Clustering

### Finding the optimal number of clusters

In [53]:
from scipy.spatial.distance import cdist
distortions = [] 
inertias = [] 
mapping1 = {} 
mapping2 = {} 
K = range(1,10) 
  
for k in K: 
    #Building and fitting the model 
    kmeanModel = KMeans(n_clusters=k).fit(RFM_Table_scaled) 
    kmeanModel.fit(RFM_Table_scaled)     
      
    distortions.append(sum(np.min(cdist(RFM_Table_scaled, kmeanModel.cluster_centers_, 
                      'euclidean'),axis=1)) / RFM_Table_scaled.shape[0]) 
    inertias.append(kmeanModel.inertia_) 
  
    mapping1[k] = sum(np.min(cdist(RFM_Table_scaled, kmeanModel.cluster_centers_, 
                 'euclidean'),axis=1)) / RFM_Table_scaled.shape[0] 
    mapping2[k] = kmeanModel.inertia_ 

In [54]:
plt.plot(K, distortions, 'bx-') 
plt.xlabel('Values of K') 
plt.ylabel('Distortion') 
plt.title('The Elbow Method using Distortion') 
plt.show() 

In [55]:
plt.plot(K, inertias, 'bx-') 
plt.xlabel('Values of K') 
plt.ylabel('Inertia') 
plt.title('The Elbow Method using Inertia') 
plt.show() 

In [56]:
# Elbow Method for K means
# Import ElbowVisualizer
from yellowbrick.cluster import KElbowVisualizer
model = KMeans()
# k is range of number of clusters.
visualizer = KElbowVisualizer(model, k=(2,12), timings= True)
visualizer.fit(RFM_Table_scaled)        # Fit data to visualizer
visualizer.show()        # Finalize and render figure

<AxesSubplot:title={'center':'Distortion Score Elbow for KMeans Clustering'}, xlabel='k', ylabel='distortion score'>

In [57]:
# Silhouette Score for K means
# Import ElbowVisualizer

from yellowbrick.cluster import KElbowVisualizer
model = KMeans()
# k is range of number of clusters.

visualizer = KElbowVisualizer(model, k=(2,12),metric='silhouette', timings= True)


visualizer.fit(RFM_Table_scaled)        # Fit the data to the visualizer
visualizer.show()        # Finalize and render the figure

<AxesSubplot:title={'center':'Silhouette Score Elbow for KMeans Clustering'}, xlabel='k', ylabel='silhouette score'>

In [59]:
# Calinski Harabasz Score for K means
# Import ElbowVisualizer
from yellowbrick.cluster import KElbowVisualizer
model = KMeans()
# k is range of number of clusters.
visualizer = KElbowVisualizer(model, k=(2,12),metric='calinski_harabasz', timings= True)
visualizer.fit(RFM_Table_scaled)        # Fit the data to the visualizer
visualizer.show()        # Finalize and render the figure



<AxesSubplot:title={'center':'Calinski Harabasz Score Elbow for KMeans Clustering'}, xlabel='k', ylabel='calinski harabasz score'>

In [60]:
# Davies Bouldin score for K means
from sklearn.metrics import davies_bouldin_score
def get_kmeans_score(data, center):
    '''
    returns the kmeans score regarding Davies Bouldin for points to centers
    INPUT:
        data - the dataset you want to fit kmeans to
        center - the number of centers you want (the k value)
    OUTPUT:
        score - the Davies Bouldin score for the kmeans model fit to the data
    '''
    #instantiate kmeans
    kmeans = KMeans(n_clusters=center)
    # Then fit the model to your data using the fit method
    model = kmeans.fit_predict(RFM_Table_scaled)
    
    # Calculate Davies Bouldin score
    score = davies_bouldin_score(RFM_Table_scaled, model)
    
    return score

scores = []
centers = list(range(2,30))
for center in centers:
    scores.append(get_kmeans_score(RFM_Table_scaled, center))
    
plt.plot(centers, scores, linestyle='--', marker='o', color='b');
plt.xlabel('K');
plt.ylabel('Davies Bouldin score');
plt.title('Davies Bouldin score vs. K');


<b>The cluster value where this decrease in inertia value becomes constant can be chosen as the right cluster value for our data. Looking at the above elbow curve, we can choose any number of clusters between 4 to 7.</b>

In [62]:
def kmeans(normalised_df_rfm, clusters_number, original_df_rfm):
    
    kmeans = KMeans(n_clusters = clusters_number, random_state = 1)
    kmeans.fit(normalised_df_rfm)

    # Extract cluster labels
    cluster_labels = kmeans.labels_
        
    # Create a cluster label column in original dataset
    df_new = original_df_rfm.assign(Cluster = cluster_labels)
    
    # Initialise TSNE
    model = TSNE(random_state=1)
    transformed = model.fit_transform(df_new)
    
    # Plot t-SNE
    plt.title('Flattened Graph of {} Clusters'.format(clusters_number))
    sns.scatterplot(x=transformed[:,0], y=transformed[:,1], hue=cluster_labels, style=cluster_labels, palette="Set1")
    
    return df_new

In [63]:
plt.figure(figsize=(10, 10))

plt.subplot(3, 1, 1)
df_rfm_k3 = kmeans(RFM_Table_scaled, 3, RFM_Table)

plt.subplot(3, 1, 2)
df_rfm_k4 = kmeans(RFM_Table_scaled, 4, RFM_Table)

plt.subplot(3, 1, 3)
df_rfm_k5 = kmeans(RFM_Table_scaled, 5, RFM_Table)

plt.tight_layout()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

KeyError: 'default'

In [66]:

def snake_plot(normalised_df_rfm, df_rfm_kmeans, df_rfm_original):

    normalised_df_rfm = pd.DataFrame(normalised_df_rfm, 
                                       index=RFM_Table.index, 
                                       columns=RFM_Table.columns)
    normalised_df_rfm['Cluster'] = df_rfm_kmeans['Cluster']

    # Melt data into long format
    df_melt = pd.melt(normalised_df_rfm.reset_index(), 
                        id_vars=['CustomerID', 'Cluster'],
                        value_vars=['Recency', 'Frequency', 'Monetary'], 
                        var_name='Metric', 
                        value_name='Value')

    plt.xlabel('Metric')
    plt.ylabel('Value')
    sns.pointplot(data=df_melt, x='Metric', y='Value', hue='Cluster')
    
    return

In [67]:
plt.figure(figsize=(9, 9))

plt.subplot(3, 1, 1)
plt.title('Snake Plot of K-Means = 3')
snake_plot(RFM_Table_scaled, df_rfm_k3, RFM_Table)

plt.subplot(3, 1, 2)
plt.title('Snake Plot of K-Means = 4')
snake_plot(RFM_Table_scaled, df_rfm_k4, RFM_Table)

plt.subplot(3, 1, 3)
plt.title('Snake Plot of K-Means = 5')
snake_plot(RFM_Table_scaled, df_rfm_k5, RFM_Table)

plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

KeyError: 'default'

<b>From the flattened graphs and the snake plots it is evident that having a cluster value of 4,segments our customers well. 
   We could also go for higher number of clusters, it completely depends on how the company wants to segment their customers.</b>

### Interpretation of the clusters formed using k-means .
 

In [67]:
def rfm_values(df):

    df_new = df.groupby(['Cluster']).agg({
        'Recency': 'mean',
        'Frequency': 'mean',
        'Monetary': ['mean', 'count']
    }).round(0)
    
    return df_new

In [68]:
rfm_values(df_rfm_k3)

Recency Frequency  Monetary      
           mean      mean      mean count
Cluster                                  
0         520.0       2.0   64219.0   344
1         416.0       4.0  142263.0    70
2         407.0      12.0  794517.0   423

In [69]:
rfm_values(df_rfm_k4)

Recency Frequency   Monetary      
           mean      mean       mean count
Cluster                                   
0         594.0       2.0    84904.0   181
1         405.0      15.0  1084065.0   293
2         425.0       4.0    85916.0   293
3         416.0       4.0   142263.0    70

In [70]:
rfm_values(df_rfm_k5)

Recency Frequency   Monetary      
           mean      mean       mean count
Cluster                                   
0         435.0       6.0   316275.0   248
1         610.0       2.0    72810.0   155
2         388.0      18.0  1353537.0   195
3         416.0       4.0   142263.0    70
4         432.0       3.0    26690.0   169